In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

import warnings
warnings.filterwarnings('ignore')

In [4]:
df_train = pd.read_csv('data/train.csv')
df_train.isna().sum().sum()
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [429]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [430]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Drop cabin, name, passengerid, age, ticket for baseline**

Can try imputing age

In [431]:
df_train.drop(columns=['Cabin', 'Name', 'PassengerId', 'Ticket'], inplace=True)

In [432]:
df_train.isna().sum().sum()

179

In [433]:
df_train['Age'].fillna(df_train['Age'].mean(), inplace=True)

In [434]:
df_train.isna().sum().sum()

2

In [435]:
df_train.dropna(inplace=True)

In [436]:
df_train.shape

(889, 8)

In [437]:
cats = ['Sex', 'Embarked']

In [438]:
ohe = OneHotEncoder(sparse=False)
ohe_cats = ohe.fit_transform(df_train[cats])

In [439]:
cats_df = pd.DataFrame(ohe_cats, columns=ohe.get_feature_names())

In [440]:
df_train.drop(columns=['Sex', 'Embarked'], inplace=True)

In [441]:
new_train_df = pd.concat([df_train, cats_df], axis=1)
new_train_df

,Survived,Pclass,Age,SibSp,Parch,Fare,x0_female,x0_male,x1_C,x1_Q,x1_S
0,0.0,3.0,22.000000,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.000000,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1.0,3.0,26.000000,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.000000,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.000000,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,2.0,27.000000,0.0,0.0,13.0000,1.0,0.0,0.0,0.0,1.0
887,1.0,1.0,19.000000,0.0,0.0,30.0000,0.0,1.0,1.0,0.0,0.0
888,0.0,3.0,29.699118,1.0,2.0,23.4500,0.0,1.0,0.0,1.0,0.0
889,1.0,1.0,26.000000,0.0,0.0,30.0000,NaN,NaN,NaN,NaN,NaN


In [442]:
new_train_df.dropna(inplace=True)

In [443]:
new_train_df.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare,x0_female,x0_male,x1_C,x1_Q,x1_S
Survived,1.000000,-0.333582,-0.074208,-0.033832,0.083536,0.255273,0.023116,-0.023116,0.010614,0.037671,-0.033024
Pclass,-0.333582,1.000000,-0.329226,0.081421,0.016458,-0.548862,-0.032381,0.032381,-0.080982,0.033399,0.049936
Age,-0.074208,-0.329226,1.000000,-0.232003,-0.178342,0.088709,0.002340,-0.002340,0.031137,0.020909,-0.040453
SibSp,-0.033832,0.081421,-0.232003,1.000000,0.414244,0.160664,-0.027617,0.027617,0.007137,-0.074180,0.040457
Parch,0.083536,0.016458,-0.178342,0.414244,1.000000,0.217332,-0.045622,0.045622,-0.001415,-0.072021,0.046592
Fare,0.255273,-0.548862,0.088709,0.160664,0.217332,1.000000,0.024440,-0.024440,0.053277,-0.021442,-0.033187
x0_female,0.023116,-0.032381,0.002340,-0.027617,-0.045622,0.024440,1.000000,-1.000000,0.083793,0.074760,-0.120507
x0_male,-0.023116,0.032381,-0.002340,0.027617,0.045622,-0.024440,-1.000000,1.000000,-0.083793,-0.074760,0.120507
x1_C,0.010614,-0.080982,0.031137,0.007137,-0.001415,0.053277,0.083793,-0.083793,1.000000,-0.149037,-0.782483
x1_Q,0.037671,0.033399,0.020909,-0.074180,-0.072021,-0.021442,0.074760,-0.074760,-0.149037,1.000000,-0.499100


In [444]:
new_train_df

,Survived,Pclass,Age,SibSp,Parch,Fare,x0_female,x0_male,x1_C,x1_Q,x1_S
0,0.0,3.0,22.000000,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.000000,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1.0,3.0,26.000000,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.000000,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.000000,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
884,0.0,3.0,25.000000,0.0,0.0,7.0500,0.0,1.0,0.0,0.0,1.0
885,0.0,3.0,39.000000,0.0,5.0,29.1250,1.0,0.0,0.0,0.0,1.0
886,0.0,2.0,27.000000,0.0,0.0,13.0000,1.0,0.0,0.0,0.0,1.0
887,1.0,1.0,19.000000,0.0,0.0,30.0000,0.0,1.0,1.0,0.0,0.0


**Test data**

In [445]:
df_test = pd.read_csv('data/test.csv')

In [446]:
df_test_sub = df_test.copy()

In [447]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [448]:
df_test['Age'].fillna(df_test['Age'].mean(), inplace=True)

In [449]:
df_test.isna().sum().sum()

328

In [450]:
df_test['Fare'].unique()

array([  7.8292,   7.    ,   9.6875,   8.6625,  12.2875,   9.225 ,
         7.6292,  29.    ,   7.2292,  24.15  ,   7.8958,  26.    ,
        82.2667,  61.175 ,  27.7208,  12.35  ,   7.225 ,   7.925 ,
        59.4   ,   3.1708,  31.6833,  61.3792, 262.375 ,  14.5   ,
        61.9792,  30.5   ,  21.6792,  31.5   ,  20.575 ,  23.45  ,
        57.75  ,   8.05  ,   9.5   ,  56.4958,  13.4167,  26.55  ,
         7.85  ,  13.    ,  52.5542,  29.7   ,   7.75  ,  76.2917,
        15.9   ,  60.    ,  15.0333,  23.    , 263.    ,  15.5792,
        29.125 ,   7.65  ,  16.1   ,  13.5   ,   7.725 ,  21.    ,
         7.8792,  42.4   ,  28.5375, 211.5   ,  25.7   ,  15.2458,
       221.7792,  10.7083,  14.4542,  13.9   ,   7.775 ,  52.    ,
         7.7958,  78.85  ,   7.8542,  55.4417,   8.5167,  22.525 ,
         7.8208,   8.7125,  15.0458,   7.7792,  31.6792,   7.2833,
         6.4375,  16.7   ,  75.2417,  15.75  ,   7.25  ,  23.25  ,
        28.5   ,  25.4667,  46.9   , 151.55  ,  18.    ,  51.8

In [451]:
df_test['Fare'].loc[df_test['Name']== 'Storey, Mr. Thomas'] = 7.5

In [452]:
df_test.loc[df_test['Name']== 'Storey, Mr. Thomas']

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,7.5,NaN,S


In [453]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [454]:
df_test.drop(columns=['Cabin', 'Name', 'PassengerId', 'Ticket'], inplace=True)

In [455]:
ohe_test = OneHotEncoder(sparse=False)
ohe_cats_test = ohe_test.fit_transform(df_test[cats])

In [456]:
test_cats_df = pd.DataFrame(ohe_cats_test, columns=ohe_test.get_feature_names())

In [457]:
df_test.drop(columns=['Sex', 'Embarked'], inplace=True)

In [458]:
new_test_df = pd.concat([df_test, test_cats_df], axis=1)
new_test_df

,Pclass,Age,SibSp,Parch,Fare,x0_female,x0_male,x1_C,x1_Q,x1_S
0,3,34.50000,0,0,7.8292,0.0,1.0,0.0,1.0,0.0
1,3,47.00000,1,0,7.0000,1.0,0.0,0.0,0.0,1.0
2,2,62.00000,0,0,9.6875,0.0,1.0,0.0,1.0,0.0
3,3,27.00000,0,0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3,22.00000,1,1,12.2875,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
413,3,30.27259,0,0,8.0500,0.0,1.0,0.0,0.0,1.0
414,1,39.00000,0,0,108.9000,1.0,0.0,1.0,0.0,0.0
415,3,38.50000,0,0,7.2500,0.0,1.0,0.0,0.0,1.0
416,3,30.27259,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [459]:
new_test_df

,Pclass,Age,SibSp,Parch,Fare,x0_female,x0_male,x1_C,x1_Q,x1_S
0,3,34.50000,0,0,7.8292,0.0,1.0,0.0,1.0,0.0
1,3,47.00000,1,0,7.0000,1.0,0.0,0.0,0.0,1.0
2,2,62.00000,0,0,9.6875,0.0,1.0,0.0,1.0,0.0
3,3,27.00000,0,0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3,22.00000,1,1,12.2875,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
413,3,30.27259,0,0,8.0500,0.0,1.0,0.0,0.0,1.0
414,1,39.00000,0,0,108.9000,1.0,0.0,1.0,0.0,0.0
415,3,38.50000,0,0,7.2500,0.0,1.0,0.0,0.0,1.0
416,3,30.27259,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


**Modelling**

In [460]:
X = new_train_df.drop(columns=['Survived'])
y = new_train_df['Survived']

In [461]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Gradient Boosted**

In [462]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train,y_train)
test_preds_boost = model.predict(X_test)

In [463]:
print(accuracy_score(y_test, test_preds_boost))

0.6573033707865169


**Logreg**

In [484]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=0.5, penalty='l1', solver='liblinear')
logreg.fit(X_train,y_train)
train_preds_logreg = logreg.predict(X_train)
test_preds_logreg = logreg.predict(X_test)

print('train:', accuracy_score(y_train, train_preds_logreg))
print('test:', accuracy_score(y_test, test_preds_logreg))

train: 0.7179125528913963
test: 0.6629213483146067


In [485]:
params = {  
    "penalty": ['none', 'l2', 'l1', 'elasticnet'],
    "C": [.4,.5,.6,1],
    "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

In [466]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(logreg, params, n_jobs=-1)
grid.fit(X_train, y_train)  
print(grid.best_score_)
grid.best_params_

/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


  File "/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got elasticnet penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver

  File "/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1314, in fit
    raise ValueError("l1_ratio must be between 0 and 1;"
ValueError: l1_ratio must be between 0 and 1; got (l1_ratio=None)

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ross/op

0.7178603536110278


/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/ross/opt/anaconda3/envs/env-1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_opt

{'C': 0.5, 'penalty': 'l1', 'solver': 'liblinear'}

In [467]:
xgb = XGBClassifier(solver= 'sag', penalty= 'elasticnet', C= 1)
xgb.fit(X_train, y_train)
train_preds = xgb.predict(X_train)
test_preds = xgb.predict(X_test)

[13:15:00] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:00] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [468]:
cross_val_score(xgb, X_train, y_train)  

[13:15:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

array([0.69014085, 0.66197183, 0.74647887, 0.71126761, 0.63829787])

In [469]:
cross_val_score(xgb, X_test, y_test)  

[13:15:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

array([0.61111111, 0.75      , 0.66666667, 0.62857143, 0.68571429])

In [ ]:
print(accuracy_score(y_test, test_preds))

**Random Search**

In [ ]:
import scipy.stats as st

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(xgb, params, n_jobs=1)  
gs.fit(X_train, y_train)  
gs.best_params_

In [470]:
clf1 = xgb = XGBClassifier(solver= 'sag', penalty= 'elasticnet', C= 1)
clf2 = LogisticRegression(C=0.5, penalty='l1', solver='liblinear')
clf3 = RandomForestClassifier(criterion = 'gini', n_estimators = 22, random_state = 42)

stacking_classifier_estimators = [
    ('xgb', clf1), ('logred', clf2), ('rfc', clf3)
]
clf = StackingClassifier(
    estimators = stacking_classifier_estimators, final_estimator = clf2
)

result = cross_val_score(clf, X_train, y_train, cv = 10, scoring = 'accuracy')
print(result.mean())


[13:15:20] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:20] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:20] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

[13:15:22] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:22] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:22] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

[13:15:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

[13:15:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

[13:15:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

0.7277867203219316


In [471]:
clf.fit(X_train,y_train)

train_preds_stack = clf.predict(X_train)
test_preds_stack = clf.predict(X_test)

print('train:', accuracy_score(y_train, train_preds_stack))
print('test:', accuracy_score(y_test, test_preds_stack))

[13:15:37] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:37] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:15:37] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1631904775127/work/src/learner.cc:573: 
Parameters: { "C", "penalty", "solver" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down t

**Submission**

In [486]:
final_preds_logreg = logreg.predict(new_test_df)

In [487]:
submission = pd.DataFrame({
        'PassengerId': df_test_sub['PassengerId'],
        'Survived': final_preds})

In [488]:
submission

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [489]:
submission['Survived'].value_counts()

0.0    301
1.0    117
Name: Survived, dtype: int64

In [490]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.7 KB


In [491]:
submission['Survived'] = submission['Survived'].map(lambda x: int(x))

In [492]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [493]:
submission.to_csv('SubmissionFinalLogReg.csv', index=False)